In [1]:
import torch
import pickle

# Load sharding.pkl
with open('sharding.pkl', 'rb') as f:
    interval_dict = pickle.load(f)

In [ ]:
models = {}
interval = 25
for k in interval_dict.keys():
    start = k
    end = start + interval
    
    shard = pickle.load(open(f'models_{start}_{end}.pkl', 'rb'))
    
    for key in shard.keys():
        if models[key] is not None:
            models[key] = shard[key]
            
# Save the models dictionary to a file
with open('all_models.pkl', 'wb') as f:
    pickle.dump(models, f)